# QoE Prediction Model Building


## 1. Compare Models in Federeted Learning Scenerio


Import libraries:


In [1]:
from keras import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional, GRU
from keras.metrics import MeanSquaredError
from typing import List, Tuple
from numpy.typing import NDArray
from functools import reduce
from numpy import add
import pandas as pd
import json
import matplotlib.pyplot as plt

# weights
Layers = List[NDArray]
# weights and number of sample
ModelUpdate = Tuple[Layers, int]

2023-07-17 19:30:27.602785: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-17 19:30:27.635289: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 19:30:27.805799: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-07-17 19:30:27.807015: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-17 19:30:29.061414: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

Check if Tensorflow can detect GPU:


In [ ]:
import tensorflow as tf

tf.config.list_physical_devices()

Read dataset from file and split to samples and target:


In [8]:
def read_data(file_id) -> Tuple[pd.DataFrame, pd.Series]:
    df = pd.read_csv(
        "./data/pokemon/chunk-{}.csv".format(file_id), index_col="id"
    ).drop(["user_id"], axis=1)
    x = df.drop(["MOS"], axis=1)
    y = df["MOS"]
    return x, y

Prepare data for models initialization and evaluation:


In [9]:
x_init, y_init = read_data("init")
x_test, y_test = read_data("test")

In [10]:
x_init

,QoA_VLCresolution,QoA_VLCbitrate,QoA_VLCframerate,QoA_VLCdropped,QoA_VLCaudiorate,QoA_VLCaudioloss,QoA_BUFFERINGcount,QoA_BUFFERINGtime,QoS_type,QoS_operator,QoU_sex,QoU_age,QoU_Ustedy,QoF_begin,QoF_shift,QoF_audio,QoF_video
id,,,,,,,,,,,,,,,,,
1630,360,93.661070,12.083333,0,21.816667,0,2,1455,5,4,1,33,5,5,5,3,3
1602,360,74.690620,8.300000,0,21.833333,0,1,2885,5,4,1,33,5,3,5,3,3
1628,360,63.705845,12.250000,1,22.283333,1,1,2042,5,4,1,33,5,5,5,1,4
1625,360,629.918800,25.400000,0,44.450000,0,1,1699,5,4,1,33,5,5,5,4,5
1600,360,97.613490,11.933333,0,21.833333,0,2,2099,5,4,1,33,5,4,5,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1728,360,435.797200,24.966667,0,44.133333,0,2,2216,2,3,1,30,5,3,5,4,4
1730,360,109.510414,11.933333,0,21.833333,0,1,2708,4,4,1,48,5,3,5,2,4
1758,360,566.690500,24.666667,0,43.783333,0,1,3136,4,3,1,48,5,4,5,4,4


Define RNN models:


In [ ]:
def lstm() -> Sequential:
    model = Sequential(
        layers=[
            LSTM(128, input_shape=(17, 1)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model


def bidirectional_lstm():
    model = Sequential(
        layers=[
            Bidirectional(LSTM(128), input_shape=(17, 1)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model


def gru():
    model = Sequential(
        layers=[
            GRU(128, input_shape=(17, 1)),
            Dense(128, activation="relu"),
            Dropout(0.2),
            Dense(64),
            Dropout(0.2),
            Dense(32),
            Dropout(0.2),
            Dense(1, activation="linear"),
        ]
    )
    model.compile(optimizer="adam", loss="mse", metrics=[MeanSquaredError()])
    return model

Utilities functions for train:


In [ ]:
def train(
    model: Sequential, weights: Layers, samples: pd.DataFrame, targets: pd.Series
) -> ModelUpdate:
    model.set_weights(weights)

    model.fit(
        samples, targets, validation_split=0.05, batch_size=128, epochs=20, shuffle=True
    )

    return model.get_weights(), len(samples)

In [ ]:
def fed_avg(updates: List[ModelUpdate]) -> Layers:
    num_examples_total = sum([num_examples for _, num_examples in updates])

    weights = [
        [layer * num_examples for layer in layers] for layers, num_examples in updates
    ]

    return [
        reduce(add, layer_updates) / num_examples_total
        for layer_updates in zip(*weights)
    ]

Init models with some data:


In [ ]:
fl_models = [lstm(), bidirectional_lstm(), gru()]
model_names = ["LSTM", "Bidirectional LSTM", "GRU"]
init_weights = [
    train(model, model.get_weights(), x_init, y_init)[0] for model in fl_models
]
fl_result = {}
for name in model_names:
    fl_result[name] = []

In each round, each model will be trained with 25 dataset separately, then 25 sets of weights will be aggregated


In [ ]:
for round in range(6):
    datasets = [read_data(25 * round + i) for i in range(1, 26)]

    for name, model in zip(model_names, fl_models):
        old_weights = model.get_weights()
        local_updates = [train(model, old_weights, x, y) for (x, y) in datasets]
        new_global_weights = fed_avg(local_updates)
        model.set_weights(new_global_weights)
        result = model.evaluate(x_test, y_test)[0]
        fl_result[name].append(result)

In [ ]:
json_result = json.dumps(fl_result, indent=4)
print(json_result)

In [ ]:
plt.plot(fl_result["LSTM"], "o-r", label="lstm")
plt.plot(fl_result["Bidirectional LSTM"], "o-b", label="bidirectional lstm")
plt.plot(fl_result["GRU"], "o-g", label="gru")

x = [25 * i for i in range(1, 7)]
xi = list(range(len(x)))
plt.xticks(xi, x)

plt.xlabel("Number of Client")
plt.ylabel("MSE")

plt.legend()
plt.show()

In [ ]:
cl_model = lstm()
cl_model.set_weights(init_weights[0])

In [ ]:
cl_result = []

In [ ]:
for round in range(6):
    datasets = [read_data(i) for i in range(1, 25 * (round + 1))]
    x = pd.concat([x for x, _ in datasets], axis=0)
    y = pd.concat([y for _, y in datasets], axis=0)
    train(cl_model, init_weights[0], x, y)
    result = cl_model.evaluate(x_test, y_test)[0]
    cl_result.append(result)

In [ ]:
plt.plot(cl_result, "o-r", label="Central")
plt.plot(fl_result["LSTM"], "o-b", label="FedAvg")
x = [25 * i for i in range(1, 7)]
xi = list(range(len(x)))
plt.xticks(xi, x)

plt.xlabel("Number of Client")
plt.ylabel("MSE")

plt.legend()
plt.show()

In [ ]:
model = lstm()

In [ ]:
train(model, model.get_weights(), x_init, y_init)

In [ ]:
import tensorflowjs as tfjs

In [ ]:
init_weights = model.get_weights()
for i in range(3, 6):
    x, y = read_data(i)
    train(model, init_weights, x, y)
    tfjs.converters.save_keras_model(model, "./weights-{}".format(i))